In [1]:
'''Loading libraries'''
import pandas as pd 
import numpy as np 
from scipy.stats import gamma, dirichlet, beta, nbinom, norm
import time
import matplotlib.pyplot as plt


'''
Notes: 
1) Likelihood_F I had to add a constant to avoid log(0). Numerical problem due multiplication?
2) Fix the saving files part, memory problms 
3) Check likelihood_P
7) Track running time
8) create a function to save files/images with a labeled name 
9) track the other variables I might want to manual change to test acceptance and convergence
10) Last resource: Rewrite the priori and likelihood removing the distributions and using the equations, cutting what
i don't need. I think using like I'm using won't converge. 
11) Last resource: Use c++ if compute canada is not a hand option or convergence problems. 

'''


'''Important parameters I need to constantly change'''
k = 100
sim = 100
start_time = time.time()


In [2]:
'''Loading dataset'''
#filename = "C:\\Users\\raoki\\Documents\\GitHub\\project_spring2019\\Data\\data_final.csv"
filename = "C:\\Users\\raque\\Google Drive\\SFU\\Project 2 - Spring 2019\\Data\\data_final.csv"
#filename = "C:\\Users\\raque\\Google Drive\\SFU\\Project 2 - Spring 2019\\Data\\data_final_sub.csv"
data = pd.read_csv(filename, sep=',')


In [3]:
'''Saving time in the first part'''

data = data.iloc[:, 0:300]
data = data.sample(n=300).reset_index(drop=True)
data.shape


(300, 300)

,gender,y,abr_ACC,abr_BLCA,abr_CHOL,abr_ESCA,abr_HNSC,abr_LGG,abr_LIHC,abr_LUSC,...,ADAM12,ADAM15,ADAM17,ADAM18,ADAM19,ADAM20,ADAM21P1,ADAM21,ADAM22,ADAM23
0,0,0,0,0,0,0,0,0,0,0,...,37,12257.0,507.0,1,505,6,5,5,123,35
1,0,1,0,0,0,0,0,0,0,0,...,2647,4326.0,3079.0,0,967,2,0,2,230,13
2,1,1,0,0,0,0,0,0,0,0,...,983,8944.0,1505.0,0,2408,3,0,1,34,155
3,0,0,0,0,0,0,0,0,0,0,...,1624,12670.0,1469.0,0,4431,3,0,8,454,107
4,1,0,0,0,0,0,0,0,0,0,...,813,3998.0,284.0,0,575,3,0,1,1,18


In [5]:
'''Organizing columns names'''
lr = data.columns[[2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]]
y = data.columns[3]
remove = data.columns[[0,1]]
data_complete = data.copy()

In [6]:
'''
Class to work with model parameters
I thought about using the default values as chain starting values, 
however, i encouter problems to change the size of arrays and matrices 
according with my currently k
'''
class parameters:
    def __init__(self, latent_v,latent_sk,latent_pj,latent_phi ,latent_tht, prediction):
        self.ln = latent_v #array with parameters that are only one number [0-c0,1-gamma0,2-eta,3-cj] 
        self.la_sk = latent_sk #array 
        self.la_pj = latent_pj #array
        self.lm_phi = latent_phi #matrix (jk)
        self.lm_tht = latent_tht #matrix  (kv)      
        self.p = prediction #array [intercept, gender, 15 cancer types, k genes]


In [7]:
'''
Declaring my priori values: param.ln = [0-c0,1-gamma0,2-eta,3-cj], param.la_sk, param.la_pj, 
param.lm_phi, param.lm_tht and param.p = [] 
#j=sample of patients 
#v=genes
#k=latent
#UPDATE PARAM 
'''
def priori_l(param):
    k = len(param.la_sk)
    j = len(param.la_pj)
    v = param.lm_phi.shape[0]
    #Prioris on the K plate
    s0 = 2.4 
    t0 = 2.4 
    c0 = gamma.pdf(x=param.ln[0],a = s0,scale = t0) #mean 6
    a0 = 2.4 
    b0 = 2.4 
    gamma0 = gamma.pdf(x=param.ln[1],a = a0,scale = b0) #mean 6
    sk = gamma.pdf(x=param.la_sk,a=param.ln[1],scale=param.ln[0])#array(k)  mean = 40
    s0 = 1
    w0 = 1/5
    eta = gamma.pdf(x=param.ln[2], a = s0, scale = w0) #mean = 5
    phivk = dirichlet.pdf(x=param.lm_phi, alpha=np.repeat(param.ln[2],v)) #MATRIX #1
    priorik = np.log(c0)+np.log(gamma0)+np.log(sk).sum()+np.log(eta)+np.log(phivk).sum()
    #priories on the J plate
    a0 = 2 
    b0 = 2 
    pj = beta.pdf(x=param.la_pj,a=a0,b=b0)  #beta(2,2) is symetric and centred in 0.5
    e0 = 6.4
    f0 = 6.4
    cj = gamma.pdf(param.ln[3], a = e0, scale = f0) #mean is 40
    priorij = np.log(pj).sum()+np.log(cj) #array    
    #prioris on K and J plate
    thetakj = gamma.pdf(x=np.transpose(param.lm_tht),a=param.la_sk.reshape(1,k),scale = param.ln[3]) #MATRIX 
    priorikj = np.log(thetakj).sum()
    return (priorik+priorij+priorikj)
  
#increase the sd to decrease the .sum()//increase uncertainity about parameters    
def priori_p(param):
    '''
    #add prioris of logistic regression 
    #features are the theta_kj parameters and clinical info
    #one parameter for each latent variable + one parameter for each clinical features
    #increased sd to decrease priori weight
    '''
    k = len(param.la_sk)
    mean = [-k*1600,1] #intercept and gender
    sd = [100,5]
    mean.extend(np.repeat(1, 15))
    sd.extend(np.repeat(5,15))
    mean.extend(np.repeat(1,k))
    sd.extend(np.repeat(5,k))
    lr = norm.pdf(param.p,loc = mean, scale = sd) #loc/mean, scale/sd
    return (np.log(lr).sum()) 

In [8]:
'''
Declare all the likelihood distributions in two separete functions
#_F represents the factor model likelihood and _P the prediction model likelihood
'''

#PASS DATA ONLY WITH RELEVANT COLUMNS FOR THIS PROBLEM 
def likelihood_F(param,dataf):
    '''phi(jk) x theta(kv) must be matrix , final dim is j patients x v genes '''
    phi_theta = np.dot(param.lm_phi, param.lm_tht)
    phi_theta = np.transpose(phi_theta)
    la_pj = np.repeat(param.la_pj, dataf.shape[1]).reshape(param.la_pj.shape[0],dataf.shape[1])
    nvj = nbinom.pmf(dataf,n = phi_theta,p= la_pj) #matrix and array?
    nvj += 0.000001  #LOOK FOR BETTER SOLUTION 
    return np.log(nvj).sum()


#old
def likelihood_P(param, datap,y):
    '''[intercept, gender, 15 cancer types, k genes]'''
    xw = param.p[0]
    print('datap', datap.head())
    for i in np.arange(1,datap.shape[1]):
        xw = xw+param.p[i]*datap[datap.columns[i-1]]
        print(xw[0],param.p[i],datap[0,datap.columns[i-1]])

    
    theta = np.transpose(param.lm_tht)
    aux = datap.shape[1]
    print('xw value:',xw.sum(), 'shape',xw.shape)
    
    for j in np.arange(0,theta.shape[1]):
        xw = xw + param.p[aux+j]*theta[:,j]
    
    #xw values are too big and np.exp is inf 
    #xwy = 0
    #for k in np.arange(1,len(y)): 
    #    xwy = xwy+xw[k]*y[k]
    # 
    #lxp = sum(-np.log(1+np.exp(xw)))
    #page 2 http://www.medicine.mcgill.ca/epidemiology/joseph/courses/EPIB-621/bayeslogit.pdf
    #http://www.utstat.utoronto.ca/reid/sta2201s/2014/feb14-annotated-copy.pdf
    #https://www.statlect.com/fundamentals-of-statistics/logistic-model-maximum-likelihood
    return 1#xwy + lxp

In [9]:
'''
Definition of the posterior distribution and the split of the datasets
Note: the parameters are ok 
'''
def posterior(param,data_F,data_P,y,k):
    j = data.shape[0]
    lf = likelihood_F(param,data_F)
    lp = likelihood_P(param, data_P,y)
    pl = priori_l(param)
    pp = priori_p(param)
    #print(lf,lp,pl,pp)
    return (lf+lp+pl+pp)

In [10]:
'''
Proposal distribution
'''

#Proposal values for the parameters related to the factor model 
#Repete the parameters related to prediction and only propose
#new values for the factor analysys part
#output is the parameters class 
def proposal_f(current):
    new = parameters(np.random.normal(current.ln,0.05), 
                     np.random.normal(current.la_sk,2),
                     np.random.beta(current.la_pj/2,current.la_pj/2),
                     np.random.normal(current.lm_phi,0.000005), #remmeber that lm_phi sum up 1 in the line (genes)
                     np.random.normal(current.lm_tht,50), #remember the average value is 1600
                     current.p)
    #phi and tht can't be negative 
    new.lm_phi[new.lm_phi<0] = 0.0000001 #this number needs to be smaller 
    col_sums = new.lm_phi.sum(axis=0)
    new.lm_phi = new.lm_phi / col_sums[np.newaxis,:]
    new.lm_tht[new.lm_tht<0]=0
    return new

#Proposal values for the parameters related to logistic regression 
#Repete the parameters related to factor analysis part and propose
#new values for the logistc regression parameters 
#output is the parameters class 
def proposal_p(current):
    new = parameters(current.ln, current.la_sk, current.la_pj, 
                     current.lm_phi, current.lm_tht, 
                     np.random.normal(current.p,10))
    return new

In [11]:
'''
Creatint the MCMC for the model
MCMC(
startvalue = initial value for the parameters
iterations = 
data = complete data with all columns 
k = number of latent variables
remove, lr, y = columns names to be removed, presente only in the logistic regression part and y
)
'''
def MCMC(startvalue, iterations, data,k, lr,y):
    '''Splitting dataset'''
    data_P = data[lr]
    data_F = data.drop(lr,axis = 1)
    data_F = data_F.drop(y,axis = 1)
    y = data[y]
    a_P = 0
    a_F = 0
    '''
    Initialization of the chains
    Note: chain_f has elements from chain_p and vice-versa. Take care to not use incorrectly
    '''     
    chain_f = []
    chain_p = []
    chain_f.append(startvalue)
    chain_p.append(startvalue)
     
    for i in np.arange(1,iterations):
        '''Factor Analysis - Latent Features'''
        #use chain_f or chain_p don't make difference here because
        #the only parameters changed are the logistic regression and 
        #they aren't used in the factor analysis part. 
        param_new_f = proposal_f(chain_f[i-1])
        param_cur_f = chain_f[i-1] 
        if i%10 == 0: 
            print('iteration ',i,' acceptance ', a_F*10/i,'-', a_P*10/i)
        prob_f = np.exp(posterior(param_new_f,data_F,data_P,y,k)-posterior(param_cur_f,data_F,data_P,y,k))
        if np.random.uniform(0,1,1)<prob_f:
            chain_f.append(param_new_f)
            a_F+=1
        else:
            chain_f.append(param_cur_f) 
        '''Logistic Regression - Prediction'''
        #chain_f[i] has the most update latent parameters and haven't changed the 
        #prediction parameters from [i-1] iteration
        param_new_p = proposal_p(chain_f[i-1])
        param_cur_p = chain_f[i-1]
    
        prob_p = np.exp(posterior(param_new_p,data_F,data_P,y,k)-posterior(param_cur_p,data_F,data_P,y,k))
        if np.random.uniform(0,1,1)<prob_p:
            chain_p.append(param_new_p)
            a_P+=1
        else:
            chain_p.append(param_cur_p)     

    return chain_p, chain_f, a_P, a_F
        
        

In [12]:
'''Non informative prioris: dirichlet has only 1, gamma distribution with 1 average, etc'''
aux = len(lr)+1
data = data.drop(remove,axis = 1)

start = parameters([k*1600,6,5,41], #ln 
                   np.repeat(15,k), #la_sk
                   np.repeat(0.5,data.shape[0]), #la_pj
                   np.repeat(1/(data.shape[1]-aux),(data.shape[1]-aux)*k).reshape((data.shape[1]-aux),k) ,#lm_phi v x k 
                   np.repeat(1600,(data.shape[0])*k).reshape(k,(data.shape[0])), #lm_theta k x j
                   np.concatenate(([-(k+aux)], np.repeat(1,k+aux-1))))  #p, k+aux-1  because intercept is already counted



In [38]:
def likelihood_P(param, datap,y):
    '''[intercept, gender, 15 cancer types, k genes]'''
    xw = param.p[0]
    for i in np.arange(1,datap.shape[1]):
        xw = param.p[i]*datap[datap.columns[i-1]]+xw
    
    theta = np.transpose(param.lm_tht)
    aux = datap.shape[1]
    print('xw value:',xw.sum(), 'shape',xw.shape)
    
    for j in np.arange(0,theta.shape[1]):
        xw = xw + param.p[aux+j]*theta[:,j]
        #if j%10 == 0:
            #print(xw[0],param.p[aux+j], theta[0,j])
    
    #xw values are too big and np.exp is inf 
    #xwy = 0
    #for k in np.arange(1,len(y)): 
    #    xwy = xwy+xw[k]*y[k]
    # 
    #lxp = sum(-np.log(1+np.exp(xw)))
    #page 2 http://www.medicine.mcgill.ca/epidemiology/joseph/courses/EPIB-621/bayeslogit.pdf
    #http://www.utstat.utoronto.ca/reid/sta2201s/2014/feb14-annotated-copy.pdf
    #https://www.statlect.com/fundamentals-of-statistics/logistic-model-maximum-likelihood
    return 1#xwy + lxp

In [39]:
#data = data.drop(remove,axis=1)

start_time = time.time()
output_p, output_f, acept_P,acept_F = MCMC(start,sim,data,k,lr,y)
end_time = time.time() - start_time

print("--- %s seconds ---" % (time.time() - start_time))



xw value: -34708 shape (300,)
1513.257101363018 1 1629.257101363018
17497.55912059168 1 1594.9487040424601
33438.49414057403 1 1581.740487066399
49437.2528091814 1 1599.9424379143022
65553.81344538186 1 1640.7728102870594
81523.06127689128 1 1570.5486802892447
97655.25229079733 1 1671.3755116179636
113635.47401725905 1 1626.6919975693183
129753.80625686153 1 1633.2370510806254
145968.94555770527 1 1560.1226564726335
xw value: -34708 shape (300,)
1484 1 1600
17484 1 1600
33484 1 1600
49484 1 1600
65484 1 1600
81484 1 1600
97484 1 1600
113484 1 1600
129484 1 1600
145484 1 1600
xw value: -34709.576933535674 shape (300,)
1502.0026428477454 1.0112450977916025 1600
17584.407945062238 0.9520427982216612 1600
32984.408991066935 0.9757990979956468 1600
48959.21510735909 1.088141099173645 1600
64936.824121964855 1.0956525303693299 1600
81191.23917416224 1.0427901472523389 1600
97043.32173383154 1.013634506948784 1600
113317.89255456216 0.9969463004859307 1600
129391.41295629872 1.085455347943103

C:\Users\raque\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\raque\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in less
C:\Users\raque\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\raque\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in less


 -34708 shape (300,)
1484 1 1600
17484 1 1600
33484 1 1600
49484 1 1600
65484 1 1600
81484 1 1600
97484 1 1600
113484 1 1600
129484 1 1600
145484 1 1600
xw value: -34708 shape (300,)
1433.355906714034 1 1549.355906714034
17244.520134872666 1 1570.3660558415872
33235.88103614021 1 1572.3899741060252
49292.327853523166 1 1579.9118588017107
65116.33895034158 1 1577.0821611169868
80892.57672158479 1 1578.1501911960327
96933.25280946116 1 1577.5296541841049
112924.48777641308 1 1643.9225749807454
128945.41758567223 1 1546.750461040598
145110.34072448077 1 1671.64271973845
xw value: -34708 shape (300,)
1484 1 1600
17484 1 1600
33484 1 1600
49484 1 1600
65484 1 1600
81484 1 1600
97484 1 1600
113484 1 1600
129484 1 1600
145484 1 1600
xw value: -34730.61374653798 shape (300,)
1455.251535107927 0.9820955709347123 1600
17343.790030839722 0.9402633797558996 1600
33549.8622634714 1.0337942650139802 1600
49267.24150199833 1.013498608361561 1600
65280.045209666765 1.0459077409925386 1600
81210.896382

xw value: -34708 shape (300,)
1484 1 1600
17484 1 1600
33484 1 1600
49484 1 1600
65484 1 1600
81484 1 1600
97484 1 1600
113484 1 1600
129484 1 1600
145484 1 1600
xw value: -34725.99413443572 shape (300,)
1399.789711496564 0.9473593848547393 1600
16931.743339547425 1.0213918400027775 1600
32512.686726297674 0.9409188011542167 1600
48404.46667807754 1.0336252222823812 1600
64347.548648753625 0.9376632114202555 1600
80553.94541475474 0.9705340184972936 1600
96299.85818507656 0.9753839048151729 1600
112415.18478381756 0.9388037038522344 1600
128531.63283674732 0.961262433102443 1600
144396.55059466115 0.9522540373716566 1600
xw value: -34708 shape (300,)
1484 1 1600
17484 1 1600
33484 1 1600
49484 1 1600
65484 1 1600
81484 1 1600
97484 1 1600
113484 1 1600
129484 1 1600
145484 1 1600
iteration  10  acceptance  0.0 - 0.0
xw value: -34708 shape (300,)
1504.1086714386195 1 1620.1086714386195
17550.441428852682 1 1586.237633476696
33528.56408023783 1 1545.3377618399836
49715.96837044145 1 1623

C:\Users\raque\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in log


...

xw value: -34708 shape (300,)
1484 1 1600
17484 1 1600
33484 1 1600
49484 1 1600
65484 1 1600
81484 1 1600
97484 1 1600
113484 1 1600
129484 1 1600
145484 1 1600
xw value: -34677.060670134735 shape (300,)
1394.3490068856552 0.9438917425940871 1600
17439.058855309726 1.0106816934628005 1600
33633.80261545918 0.9974140754431705 1600
49607.9676802071 1.0733154415237314 1600
65552.11661453173 1.04208125454598 1600
81231.53093980733 0.9535862706574812 1600
97156.11343957976 1.0010035751222657 1600
113135.32491286403 0.9371523195124081 1600
129030.58295380247 0.9453762714648708 1600
145045.59712984928 1.0338718835127583 1600
xw value: -34708 shape (300,)
1484 1 1600
17484 1 1600
33484 1 1600
49484 1 1600
65484 1 1600
81484 1 1600
97484 1 1600
113484 1 1600
129484 1 1600
145484 1 1600
xw value: -34708 shape (300,)
1502.0132464512728 1 1618.0132464512728
17430.341113094346 1 1647.8858585884968
33482.41849520028 1 1647.9862385931833
49336.62847427891 1 1613.802094649203
65589.86538690083 1 1597

129484 1 1600
145484 1 1600
xw value: -34722.69933095499 shape (300,)
1496.2043901906477 1.0076672826012836 1600
17517.142827663276 0.9838250256370582 1600
33474.9733827992 1.064081166695763 1600
49438.20178237445 0.9464813593788622 1600
65450.67522379463 0.9212146731317152 1600
81247.7745946683 1.0388423006685161 1600
97413.58740298155 1.0327246156435366 1600
113328.69590542583 0.9878584710114106 1600
129022.88665626805 0.9805988329560367 1600
145031.9012838942 1.0337433152401054 1600
xw value: -34708 shape (300,)
1484 1 1600
17484 1 1600
33484 1 1600
49484 1 1600
65484 1 1600
81484 1 1600
97484 1 1600
113484 1 1600
129484 1 1600
145484 1 1600
xw value: -34708 shape (300,)
1519.1782226995108 1 1635.1782226995108
17498.55631270836 1 1645.2245280721384
33577.111157298095 1 1622.7419500976164
49490.63343024861 1 1695.3721637659673
65590.00913641727 1 1675.955054671315
81333.5473149636 1 1577.481484554739
97125.50701162874 1 1634.533190413154
112988.84696176084 1 1684.5275790013889
128636

xw value: -34708 shape (300,)
1484 1 1600
17484 1 1600
33484 1 1600
49484 1 1600
65484 1 1600
81484 1 1600
97484 1 1600
113484 1 1600
129484 1 1600
145484 1 1600
xw value: -34715.86891695351 shape (300,)
1496.9860387504468 1.0081204939519943 1600
17092.287258328142 0.9558384227144167 1600
33266.93848221864 0.9946045366017204 1600
49092.14273747172 0.9397868794822561 1600
65082.40833063893 0.983390379839788 1600
81251.77426334766 1.0535595875908927 1600
97237.15889171236 0.982532005495604 1600
113141.10963220672 1.0365483828888178 1600
129004.60909278489 0.995522210380542 1600
144852.299426188 1.0188136048736298 1600
xw value: -34708 shape (300,)
1484 1 1600
17484 1 1600
33484 1 1600
49484 1 1600
65484 1 1600
81484 1 1600
97484 1 1600
113484 1 1600
129484 1 1600
145484 1 1600
xw value: -34708 shape (300,)
1470.3554393384138 1 1586.3554393384138
17483.10695772866 1 1696.818737269885
33598.08556065949 1 1566.4744979163538
49579.206073447116 1 1562.570995155204
65902.21356170396 1 1661.118

17484 1 1600
33484 1 1600
49484 1 1600
65484 1 1600
81484 1 1600
97484 1 1600
113484 1 1600
129484 1 1600
145484 1 1600
xw value: -34696.16016471662 shape (300,)
1461.4351955855439 0.9858713145354722 1600
17589.16959805867 1.0077739203011506 1600
33730.15269395079 1.0147140340780585 1600
50315.39156121392 1.0632325333842967 1600
66390.89247918731 1.0276153700859603 1600
82279.59489372342 0.9920510002635011 1600
98320.24592753797 1.0345935621343691 1600
114230.34830437001 1.0622106632783925 1600
130441.92102530565 1.0795724486855662 1600
146574.5631537451 1.0148861852959543 1600
xw value: -34708 shape (300,)
1484 1 1600
17484 1 1600
33484 1 1600
49484 1 1600
65484 1 1600
81484 1 1600
97484 1 1600
113484 1 1600
129484 1 1600
145484 1 1600
xw value: -34708 shape (300,)
1563.5826683769992 1 1679.5826683769992
17750.493356669627 1 1517.2491722483373
33651.57394148981 1 1513.174467695534
49547.439400784526 1 1530.184631998733
65442.98134596751 1 1617.9914502592314
81497.88395711186 1 1661.21

145268.54930902197 0.9586202292502377 1600
xw value: -34708 shape (300,)
1484 1 1600
17484 1 1600
33484 1 1600
49484 1 1600
65484 1 1600
81484 1 1600
97484 1 1600
113484 1 1600
129484 1 1600
145484 1 1600
--- 30.559593677520752 seconds ---


In [15]:
'''Organizing outputs'''

#TRANSFORM IN A FUNCTION TO STOP HAVE MEMORY PROBLEMS 

#Logistic Regression Parameters 
output_logistic = np.concatenate((output_p[0].p,output_p[1].p),axis = 0)
#Factor Model parameters
output_factor_ln = np.concatenate((output_f[0].ln,output_f[1].ln),axis = 0)
output_factor_la_sk = np.concatenate((output_f[0].la_sk,output_f[1].la_sk),axis = 0)
output_factor_la_pj = np.concatenate((output_f[0].la_pj,output_f[1].la_pj),axis = 0)
#Matrix AxB is saved as a line A*B elements 
output_factor_lm_phi= np.concatenate((output_f[0].lm_phi,output_f[1].lm_phi),axis = 0)
output_factor_lm_tht= np.concatenate((output_f[0].lm_tht,output_f[1].lm_tht),axis = 0)


for i in np.arange(2,sim):
    output_logistic = np.concatenate((output_logistic,output_p[i].p),axis = 0)
    output_factor_ln = np.concatenate((output_factor_ln,output_f[1].ln),axis = 0)
    output_factor_la_sk = np.concatenate((output_factor_la_sk,output_f[1].la_sk),axis = 0)
    output_factor_la_pj = np.concatenate((output_factor_la_pj,output_f[1].la_pj),axis = 0)

output_logistic = output_logistic.reshape(sim,len(output_p[0].p) )    
output_factor_ln = output_factor_ln.reshape(sim,len(output_f[0].ln) )
output_factor_la_sk = output_factor_la_sk.reshape(sim,len(output_f[0].la_sk) )   
output_factor_la_pj = output_factor_la_pj.reshape(sim,len(output_f[0].la_pj) )

np.savetxt('Data\\output_logistic.txt', output_logistic, delimiter=',')  
np.savetxt('Data\\output_factor_ln.txt', output_factor_ln, delimiter=',')  
np.savetxt('Data\\output_factor_la_sk.txt', output_factor_la_sk, delimiter=',')  
np.savetxt('Data\\output_factor_la_pj.txt', output_factor_la_pj, delimiter=',')  


    output_factor_lm_phi = np.concatenate((output_factor_lm_phi,output_f[1].lm_phi),axis = 0)
    output_factor_lm_tht = np.concatenate((output_factor_lm_tht,output_f[1].lm_tht),axis = 0)
    
output_factor_lm_phi = output_factor_lm_phi.reshape(sim,output_f[0].lm_phi.shape[0]*output_f[0].lm_phi.shape[1])   
output_factor_lm_tht = output_factor_lm_tht.reshape(sim,output_f[0].lm_tht.shape[0]*output_f[0].lm_tht.shape[1])

np.savetxt('Data\\output_factor_lm_phi.txt', output_factor_lm_phi, delimiter=',')  
np.savetxt('Data\\output_factor_lm_tht.txt', output_factor_lm_tht, delimiter=',')  


IndentationError: unexpected indent (<ipython-input-15-6fc577daeca5>, line 33)

In [ ]:
#acceptance and rejection  problem 
#create a easy way to identify paramerters based on name
#test several parameters at once

In [ ]:
np.exp([160,160])


In [ ]:
plt.plot(np.arange(0,len(output_logistic[:,0])),output_logistic[:,0], 'r-', alpha=1)
plt.xlabel('Logistic Regression - intercept')
plt.savefig('Data\\lr_intercept.png')

plt.plot(np.arange(0,len(output_logistic[:,1])),output_logistic[:,1], 'r-', alpha=1)
plt.xlabel('Logistic Regression - gender')
plt.savefig('Data\\lr_gender.png')

plt.plot(np.arange(0,len(output_logistic[:,10])),output_logistic[:,10], 'r-', alpha=1)
plt.xlabel('Logistic Regression - cancer type (one of them)')
plt.savefig('Data\\lr_cancertype.png')

plt.plot(np.arange(0,len(output_logistic[:,30])),output_logistic[:,30], 'r-', alpha=1)
plt.xlabel('Logistic Regression - k (one of them)')
plt.savefig('Data\\lr_k.png')

plt.plot(np.arange(0,len(output_factor_ln[:,2])),output_factor_ln[:,2], 'r-', alpha=1)
plt.xlabel('Factor Analysis - parameter')
plt.savefig('Data\\fa_1.png')

